In [5]:
using Distributions
using Optim

In [17]:
delta1, delta2 = -3 , -2
beta1, beta2 = [1 4], [2 3]
rho = -2
lambda = exp(rho)/(1+exp(rho))

N = 200
Y = zeros(N,2)
X = zeros(N,2)

io = open("monte_carlo.csv", "w")
println(io, "y1,y2,x1,x2")

for i=1:N
    x1,x2 = rand(Normal(0.0, 1.0),3)
    e1,e2 = rand(Logistic(),2)
    select = rand(Categorical([lambda, (1-lambda)]))
    
    #see logit response i.e. definitely 11
    eq = []
    for y_init=[[0 0],[1 1],[0 1],[1 0]]
        y1,y2 = sum(beta1.*[1 x1]) + delta1*y_init[2] + e1 >= 0, sum(beta2.*[1 x2]) +delta2*y_init[1] + e2 >= 0
        
        if false
            println( "$(1*y1) $(1*y2),$(y_init)" )
            println(sum(beta1.*[1 x1]) + delta1*y_init[2] + e1," ", sum(beta2.*[1 x2]) +delta2*y_init[1] + e2)
            println("-----")
        end
            
        if (y1==y_init[1]) & (y2==y_init[2])
            append!(eq,[y_init])
        end
    end
    #println( eq )
    #println( "========")
    y1, y2 = eq[1]
    if size(eq)[1] >= 2
        y1, y2 = eq[select]
    end
    Y[i,:] =[y1 y2]
    X[i,:] =[x1 x2]
    println(io, "$(y1),$(y2),$(x1),$(x2)")
end

close(io)

In [28]:
function ll_obj(params;xs=X,ys=Y)
    n = size(X)[1]
    k = Integer(size(X)[2]/2) + 1
    
    p = []
    
    for bound=[[0,0],[1,1],[0,1],[1,0]]
        util1 = params[1] .+ xs[:,1].*params[2]  .+ bound[2].*params[3]
        util2 = params[4] .+ xs[:,2].*params[5]  .+ bound[1].*params[6]
        prob = exp.(util1).^(bound[1])./(1 .+ exp.(util1))
        prob = prob .* exp.(util2).^(bound[2])./(1 .+ exp.(util2))
        append!(p,[prob])
    end
    
    p00 = (1 .- ys[:,1]).*(1 .- ys[:,2])
    p11 = ys[:,1].*ys[:,2]
    p10 = ys[:,1].* (1 .- ys[:,2])
    p01 = (1 .- ys[:,1]) .* ys[:,2]
    
    lambda = exp(params[7])/(1+exp(params[7]))
    
    ll = p00 .* p[1] + p11 .*p[2]
    ll = ll .+ p01 .* ( lambda .* p[3] + (1-lambda).*(1 .- p[1] .- p[2] .- p[4] ) )
    ll = ll .+ p10 .* ( (1-lambda) .* p[4] + lambda.*(1 .- p[1] .- p[2] .- p[3] ) )
    print(lambda)
    print(ll)
    return -sum(log.(ll))    
end


ll_obj([1 4 -3 2 3 -2 -1];xs=X,ys=Y)

0.2689414213699951[0.30757, 0.859648, 0.846886, 0.925484, 0.925858, 0.75043, 0.493286, 0.866413, 0.206261, 0.196453, 0.678524, 0.508247, 0.687809, 0.995806, 0.539487, 0.9974, 0.99884, 0.329642, 0.576633, 0.548459, 0.972413, 0.850256, 0.353994, 0.560859, 0.607969, 0.816668, 0.795395, 0.0130457, 0.430498, 0.325541, 0.630904, 0.957067, 0.278153, 0.982256, 0.689463, 0.990964, 0.725974, 0.224264, 0.912126, 0.0277992, 0.60777, 0.89665, 0.113194, 0.813206, 0.946031, 0.214895, 0.552004, 0.316971, 0.897579, 0.742444, 0.977413, 0.635136, 0.995197, 0.177529, 0.647086, 0.695328, 0.479652, 0.923726, 0.892435, 0.542178, 0.985825, 0.752956, 0.226296, 0.867816, 0.95677, 0.280355, 0.429283, 0.394525, 0.843985, 0.410504, 0.638319, 0.9966, 0.808333, 0.745544, 0.958081, 0.741468, 0.330658, 0.674573, 0.347945, 0.686672, 0.736365, 0.979486, 0.676097, 0.613291, 0.524933, 0.412693, 0.973989, 0.753823, 0.697767, 0.977462, 0.653735, 0.656084, 0.338297, 0.430357, 0.307639, 0.889244, 0.302395, 0.881901, 0.926438,

124.20582998636017

In [19]:
a = 10
b =(exp(a)/(1+exp(a)))^2 + 1/4 + 1/2*exp(a)/(1+exp(a))
println( (exp(a)/(1+exp(a)))^2 )
println( 1/2*exp(a)/(1+exp(a)) )
print(b)

0.9999092063235617
0.4999773010656488
1.7498865073892105

In [20]:
function maximum_likelihood(params)   
    return optimize(ll_obj, params, LBFGS())
end

result =  maximum_likelihood([1 4 -3 2 3 -2 -1.])
print(result.minimizer)
print(result.minimum)

[0.982616 4.53741 -2.80866 1.77774 2.65394 -1.63173 -0.809166]122.77818335284223